# Playground Season 3 Episode 8
## Problem Statement
You are hired by a company Gem Stones co ltd, which is a cubic zirconia manufacturer. You are provided with the dataset containing the prices and other attributes of almost 27,000 cubic zirconia (which is an inexpensive diamond alternative with many of the same qualities as a diamond). <br>

The company is earning different profits on different prize slots. You have to help the company in predicting the price for the stone on the basis of the details given in the dataset so it can distinguish between higher profitable stones and lower profitable stones so as to have a better profit share. Also, provide them with the best 5 attributes that are most important.

**Description of Features**
- Carat: Carat weight of the cubic zirconia.
- Cut: Describe the cut quality of the cubic zirconia. Quality is increasing order Fair, Good, Very Good, Premium, Ideal.
- Color: Colour of the cubic zirconia. With D being the best and J the worst.
- Clarity: Cubic zirconia Clarity refers to the absence of the Inclusions and Blemishes. (In order from Best to Worst, FL = flawless, I3= level 3 inclusions) FL, IF, VVS1, VVS2, VS1, VS2, SI1, SI2, I1, I2, I3
- Depth: The Height of a cubic zirconia, measured from the Culet to the table, divided by its average Girdle Diameter
- Table: The Width of the cubic zirconia's Table expressed as a Percentage of its Average Diameter.
- Price (target): The Price of the cubic zirconia.
- X: Length of the cubic zirconia in mm.
- Y: Width of the cubic zirconia in mm.
- Z: Height of the cubic zirconia in mm.

Reference: 
https://www.ganoksin.com/article/4-cs-gemstone-valuation/

### Overview

1. Understand the shape and type of data
2. Data Cleaning
3. Data Exploration
4. Feature Engineering
5. Data Preprocessing for Model
6. Basic Model Building
7. Model Tuning
8. Ensemble Model Building
9. Results

In [ ]:
# Libraries for EDA
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load Data
df_train = pd.read_csv("../input/playground-series-s3e8/train.csv")
df_test = pd.read_csv("../input/playground-series-s3e8/test.csv")

# Add the "train" column and merge the data
df_train['train'] = 1
df_test['train'] = 0
df_total = pd.concat([df_train, df_test])

### 1. Understanding the Shape and Type of Data
<i>This involves getting a high-level understanding of the data you are working with, including its size, format, and the types of variables present.</i>


In [ ]:
display(df_total.info())

In [ ]:
display(df_total.nunique())

In [ ]:
print(f"Shape of training data: {df_train.shape}")
print(f"Shape of test data: {df_test.shape}")
print(f"Shape of total data: {df_total.shape}")

In [ ]:
# What does our data look like?
display(df_train.head())

In [ ]:
# General statistics of our dataset
display(df_total.describe())

In [ ]:
# Define the target variable
target = 'price'

# Assign list of featured columns
feat_cols = ['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z']
num_cols = ['carat', 'depth', 'table', 'x', 'y', 'z']
cat_cols = ['cut', 'color', 'clarity']

### 2. Data Cleaning
<i>This involves identifying and handling missing or invalid data, dealing with outliers, and correcting any other errors or inconsistencies in the data.</i>


In [ ]:
# Find any duplicated rows in dataset
print(f"Number of duplicated rows: {df_total.duplicated().sum()}")

In [ ]:
# Find any missing values in dataset
print("Number of NaN values in each column:")
print(df_total.isna().sum())

### 3. Data Exploration
<i>This involves visualizing and summarizing the data to gain insights into its distribution, correlations, and other patterns.</i>

In [ ]:
# Pairwise plot of numerical data, groupedby color
percentOfSample = 0.1 # 10% of dataset

sns.pairplot(data = df_train.sample(frac = percentOfSample),
             vars = num_cols + [target],
             hue = 'color',
             corner = True)

In [ ]:
# Pairwise plot of numerical data, groupedby clarity
sns.pairplot(data = df_train.sample(frac = percentOfSample),
             vars = num_cols + [target],
             hue = 'clarity',
             corner = True)

In [ ]:
# Pairwise plot of numerical data, groupedby cut
sns.pairplot(data = df_train.sample(frac = percentOfSample),     
             vars = num_cols + [target],
             hue = 'cut',
             corner = True)

Depth and table do not appear to present strong correlation to price or with other features

In [ ]:
# Correlation map of Numeric Data and Target
display(pd.DataFrame(df_train[num_cols + [target]].corr()))

fig, ax = plt.subplots(figsize = (8, 6))
    
sns.heatmap(data = df_train[num_cols + [target]].corr(),
            square = True,
            vmin = -1, 
            vmax = 1, 
            annot = True,
            yticklabels = num_cols + [target], 
            xticklabels = num_cols + [target],
            annot_kws = {"size": 30 / np.sqrt(len(df_train[num_cols + [target]].corr()))},
)

Depth and table offer low correlation to price and other features. 

High corrlation with carat (one of the 4C's).

Geometric dimensions, 'x', 'y' and 'z' have high correlation with price. This is expected given the relationship with volume, density and weight (carat). 

Shape (round, pear, square, etc) may also be a factor in the price of the gemstone. We can estimate it's shape given the x, y, z, carat, table and depth but may introduce additional bias into our model.

In [ ]:
# Visualising Categorical Data

# Define order of categories for each variable
cut_order = ['Fair', 'Good', 'Very Good', 'Premium', 'Ideal']
color_order = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_order = ['I3', 'I2', 'I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF', 'FL']

# Create a 1x3 grid of subplots
fig, axs = plt.subplots(nrows = 1, ncols = 3, figsize=(15, 5))

# Bar plots
sns.barplot(x = df_train['clarity'].value_counts().index, 
            y = df_train['clarity'].value_counts(),
            order = clarity_order, ax = axs[0])

sns.barplot(x = df_train['color'].value_counts().index, 
            y = df_train['color'].value_counts(),
            order = color_order, ax = axs[1])

sns.barplot(x = df_train['cut'].value_counts().index, 
            y = df_train['cut'].value_counts(),
            order = cut_order, ax = axs[2])

# Adjust the layout of the plot
plt.tight_layout()
sns.despine(trim = True)
plt.subplots_adjust(hspace = 0.3, wspace = 0.3)
sns.set(font_scale = 0.8)
plt.show()

In [ ]:
# Create a 3x2 grid of subplots
fig, axs = plt.subplots(nrows = 3, ncols = 2, figsize = (15, 15))

# Pointplots
sns.pointplot(data = df_train, x = "color", y = "price", hue = "clarity", order = color_order, ax = axs[0,0])
sns.pointplot(data = df_train, x = "color", y = "price", hue = "cut", order = color_order, ax = axs[0,1])
sns.pointplot(data = df_train, x = "clarity", y = "price", hue = "cut", order = clarity_order, ax = axs[1,0])
sns.pointplot(data = df_train, x = "clarity", y = "price", hue = "color", order = clarity_order, ax = axs[1,1])
sns.pointplot(data = df_train, x = "cut", y = "price", hue = "clarity", order = cut_order, ax = axs[2,0])
sns.pointplot(data = df_train, x = "cut", y = "price", hue = "color", order = cut_order, ax = axs[2,1])

# Adjust the layout of the plot
plt.tight_layout()
sns.despine(trim = True)
plt.subplots_adjust(hspace = 0.3, wspace = 0.3)
sns.set(font_scale = 0.8)
plt.show()

In [ ]:
# Create a 1x3 grid of subplots
fig, axs = plt.subplots(nrows = 1, ncols = 3, figsize=(15, 5))

# Boxplots
sns.boxplot(data = df_train, x = 'clarity', y = 'price', order = clarity_order, ax = axs[0])
sns.boxplot(data = df_train, x = 'color', y = 'price', order = color_order, ax = axs[1])
sns.boxplot(data = df_train, x = 'cut', y = 'price', order = cut_order, ax = axs[2])

# Adjust the layout of the plot
plt.tight_layout()
sns.despine(trim = True)
plt.subplots_adjust(hspace = 0.3, wspace = 0.3)
sns.set(font_scale = 0.8)
plt.show()

High numbers of outliers obseverved in box plots. This is expected given earlier analysis, as shown by the correlation of carat, x, y, and z with price.

### 4. Feature Engineering
<i>This involves creating new features or transforming existing features to better represent the underlying patterns in the data.</i>

Geometric data shown to be highly correlated with price, 

So we can estimate what the shape is given its volume, table, depth, x, y, z data. But for now let's ignore this as it may introduce new bias into our model.

In [ ]:
# # Geometric data shown to be highly correlated with price
# df_total['weight_g'] = df_total['carat'] / 5                    # Converting carat to grams, units: g
# df_total['density'] = 5.65                                      # Approximate density of CZ, units: g/cm3
# df_total['volume'] = df_total['weight_g'] / df_total['density'] # volume = weight / density


### 5. Data Processing
<i>This involves scaling, encoding, and splitting the data in preparation for model building.</i>

In [ ]:
# Libraries required for preprocessing data
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split, cross_val_score, KFold

In [ ]:
print(cut_order)

In [ ]:
# Create a LabelEncoder object
oe_cut = OrdinalEncoder(categories = [cut_order])
oe_color = OrdinalEncoder(categories = [color_order])
oe_clarity = OrdinalEncoder(categories = [clarity_order])

# Encode the categorical columns using the OrdinalEncoder object
df_total['cut_oe'] = oe_cut.fit_transform(df_total[['cut']])
df_total['color_oe'] = oe_color.fit_transform(df_total[['color']])
df_total['clarity_oe'] = oe_clarity.fit_transform(df_total[['clarity']])


In [ ]:
# # Create a OneHotEncoder object and encode the categorical columns
# oh_encode = OneHotEncoder(sparse = False, handle_unknown = 'ignore')
# oh_encoded_data = oh_encode.fit_transform(df_total[cat_cols])
# oh_encoded_df = pd.DataFrame(oh_encoded_data, columns = oh_encode.get_feature_names(cat_cols))
# oh_encoded_df.reset_index(drop = True, inplace=True)

# # Merge the encoded dataframe with the original dataframe
# df_total.reset_index(drop = True, inplace=True)
# df_total = pd.concat([df_total, encoded_df], axis = 1)

We are provided categorical variables that have an inherent ordering to its value. Therefore, labelEncoder is our preferred method.

In [ ]:
# Prepare the training dataset
df_model = df_total.drop(['id','price', 'cut', 'color', 'clarity'], axis = 1)
training_values = df_model[df_model.train == 1].drop(['train'], axis = 1)
test_values = df_model[df_model.train == 0].drop(['train'], axis = 1)
training_target = df_total[df_total.train == 1].price

In [ ]:
# Split data into training and testing sets
train_X, test_X, train_y, test_y = train_test_split(training_values, training_target, test_size = 0.2)

### 6. Basic Model Building
<i>This involves selecting an appropriate algorithm and building a simple model to establish a baseline performance metric.</i>

In [ ]:
# Libraries for ML models
import xgboost as xgb
import lightgbm as lgb
import catboost as cat

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor

# Libraries to Evaluate models
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from eli5.sklearn import PermutationImportance

In [ ]:
RANDOM_STATE = 1

# Create instances of basic models using default parameters
xgb_model = XGBRegressor(random_state = RANDOM_STATE, objective = 'reg:squarederror')
lgb_model = LGBMRegressor(random_state = RANDOM_STATE, objective = 'regression')
cat_model = CatBoostRegressor(random_state = RANDOM_STATE, objective = 'RMSE', verbose = False)

voting_reg_model = VotingRegressor(estimators = [
    ('Xtreme Gradient Boosting', xgb_model),
    ('Light Gradient Boosting', lgb_model),
    ('Cat Gradient Boosting', cat_model)])

estimators = [
    ('Xtreme Gradient Boosting', xgb_model),
    ('Light Gradient Boosting', lgb_model),
    ('Cat Gradient Boosting', cat_model),
    ('Voting Regressor', voting_reg_model)]

# Evaluate each model using cross-validation and print the results
for model in estimators:
    cv_scores = cross_val_score(estimator = model[1], X = train_X, y = train_y, cv = 5, scoring = 'neg_mean_squared_error')
    rmse_scores = np.sqrt(-cv_scores)
    print(f"{model[0]} Cross-Validation Mean: {rmse_scores.mean():.2f} +/- {rmse_scores.std():.2f}\n")   

In [ ]:
# Fit Model to training data
xgb_model.fit(train_X, train_y)
lgb_model.fit(train_X, train_y)
cat_model.fit(train_X, train_y)
voting_reg_model.fit(train_X, train_y)

In [ ]:
# Get feature importance values and corresponding feature names
importance_xgb = xgb_model.feature_importances_
importance_lgb = lgb_model.feature_importances_
importance_cat = cat_model.feature_importances_

# Plot feature importance for XGBoost
sorted_idx = np.argsort(importance_xgb)
fig, ax = plt.subplots(figsize = (12, 6))
ax.barh(range(len(sorted_idx)), importance_xgb[sorted_idx], align = 'center')
ax.set_yticks(range(len(sorted_idx)))
ax.set_yticklabels(np.array(train_X.columns)[sorted_idx])
ax.set_title('Feature Importance for XGBoost')
plt.show()

# Plot feature importance for LightGBM
sorted_idx = np.argsort(importance_lgb)
fig, ax = plt.subplots(figsize = (12, 6))
ax.barh(range(len(sorted_idx)), importance_lgb[sorted_idx], align = 'center')
ax.set_yticks(range(len(sorted_idx)))
ax.set_yticklabels(np.array(train_X.columns)[sorted_idx])
ax.set_title('Feature Importance for LightGBM')
plt.show()

# Plot feature importance for CatBoost
sorted_idx = np.argsort(importance_cat)
fig, ax = plt.subplots(figsize = (12, 6))
ax.barh(range(len(sorted_idx)), importance_cat[sorted_idx], align = 'center')
ax.set_yticks(range(len(sorted_idx)))
ax.set_yticklabels(np.array(train_X.columns)[sorted_idx])
ax.set_title('Feature Importance for CatBoost')
plt.show()

XGBoost may be over emphasising 'y'

### 7. Model Tuning and Optimising Features
<i>This involves optimising the hyperparameters of the model to improve its performance on the validation set.</i>

In [ ]:
import optuna
from optuna.samplers import TPESampler

In [ ]:
# Define objective function to be minimized
def objective_xgb(trial, X = training_values, y = training_target):
    
    # Split data into training and validation sets
    train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.2)
    
    # Define hyperparameters to optimize
    params = {
        'tree_method':'gpu_hist',
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'subsample': trial.suggest_float('subsample', 0.1, 1.0),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 1.0),
        'n_estimators': 10000,
        'max_depth': trial.suggest_int('max_depth', 2, 10),
        'min_child_weight': trial.suggest_float('min_child_weight', 1, 300),
        'gamma': trial.suggest_loguniform('gamma', 1e-3, 10.0),
    }
    
    # Create model and fit to training data
    model = XGBRegressor(**params)
    model.fit(train_X, train_y, eval_set=[(test_X, test_y)], early_stopping_rounds=50, verbose=False)
    
    # Evaluate model on validation set
    preds = model.predict(test_X)
    rmse = mean_squared_error(test_y, preds, squared=False)
    
    return rmse

# Create Optuna study and optimize hyperparameters
study = optuna.create_study(direction='minimize')
study.optimize(objective_xgb, n_trials=200)

# Get best hyperparameters
best_trial_xgb = study.best_trial.params

print(f'Number of finished trials: {len(study.trials)}')
print(f'Best trial: {best_trial_xgb}')

Best trial: {'lambda': 0.027518478540242433, 'alpha': 0.032866241738019684, 'colsample_bytree': 0.5749802969702276, 'subsample': 0.8043266493482308, 'learning_rate': 0.050413311493018684, 'max_depth': 7, 'min_child_weight': 55.13650812717483, 'gamma': 0.056026172952691015}

In [ ]:
# Define objective function to be minimized
def objective_lgbm(trial, X = training_values, y = training_target):
    
    # Split data into training and validation sets
    train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.2)
    
    # Define hyperparameters to optimize
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'num_leaves': trial.suggest_int('num_leaves', 10, 1000),
        'max_depth': trial.suggest_int('max_depth', 5, 15, step = 1),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 5, 100),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.1, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.1, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 10.0),
        'verbosity': -1,
        'n_estimators': 10000,
    }
    
    # Create model and fit to training data
    model = LGBMRegressor(**params)
    model.fit(train_X, train_y, eval_set=[(test_X, test_y)], early_stopping_rounds=50, verbose=False)
 
    # Calculate RMSE on validation set
    preds = model.predict(test_X)
    rmse = mean_squared_error(test_y, preds, squared = False)
    
    # Return RMSE as the objective value to be minimized
    return rmse

# Define study and optimize hyperparameters
study = optuna.create_study(direction = 'minimize')
study.optimize(objective_lgbm, n_trials = 200)

# Get best hyperparameters and corresponding RMSE
best_trial_lgb = study.best_trial.params

print(f'Number of finished trials: {len(study.trials)}')
print(f'Best trial: {best_trial_lgb}')

Number of finished trials: 200
Best trial: {'num_leaves': 771, 'max_depth': 6, 'min_data_in_leaf': 90, 'learning_rate': 0.08125313292346216, 'feature_fraction': 0.8653153559394122, 'bagging_fraction': 0.6625570556339628, 'bagging_freq': 10, 'min_child_samples': 94, 'reg_alpha': 2.5483602114471396e-06, 'reg_lambda': 0.01891811952141336}

In [ ]:
# Define objective function to be minimized
def objective_cat(trial, X = training_values, y = training_target):
    
    # Split data into training and validation sets
    train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2)
    
    # Define hyperparameters to optimize
    params = {
        'n_estimators': 10000,
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10.0),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 100.0),
        'random_strength': trial.suggest_float('random_strength', 0.0, 10.0),
        'one_hot_max_size': trial.suggest_int('one_hot_max_size', 2, 16),
        'cat_features': [i for i in range(len(X.columns)) if X.dtypes[i] == 'category'],
        'verbose': False,
        'allow_writing_files': False,
        'thread_count': -1
    }
    
    # Create model and fit to training data
    model = CatBoostRegressor(**params)
    model.fit(train_X, train_y, eval_set = [(test_X, test_y)], early_stopping_rounds=100, verbose=False)
    
    # Evaluate model on validation set
    preds = model.predict(test_X)
    rmse = mean_squared_error(test_y, preds, squared = False)
    
    return rmse

# Create Optuna study and optimize hyperparameters
study = optuna.create_study(direction = 'minimize')
study.optimize(objective_cat, n_trials = 200)

# Get best hyperparameters
best_trial_cat = study.best_trial.params
print(f'Number of finished trials: {len(study.trials)}')
print(f'Best trial: {best_trial_cat}')


Best trial: {'learning_rate': 0.06885310731231294, 'depth': 6, 'l2_leaf_reg': 2.0974907315651783, 'bagging_temperature': 59.04421296914569, 'random_strength': 4.0504687744470225, 'one_hot_max_size': 11}

### 8. Ensemble Model Building 
<i>This involves combining multiple models to improve overall predictive performance.</i>

In [ ]:
# Create an instance of the XGBRegressor with the best trial hyperparameters
xgb_model = XGBRegressor(**best_trial_xgb)
lgb_model = LGBMRegressor(**best_trial_lgb)
cat_model = CatBoostRegressor(**best_trial_cat)

voting_reg_model = VotingRegressor(estimators = [
    ('Xtreme Gradient Boosting', xgb_model),
    ('Light Gradient Boosting', lgb_model),
    ('Cat Gradient Boosting', cat_model)])

estimators = [
    ('Xtreme Gradient Boosting', xgb_model),
    ('Light Gradient Boosting', lgb_model),
    ('Cat Gradient Boosting', cat_model),
    ('Voting Regressor', voting_reg_model)]

# Evaluate each model using cross-validation and print the results
for model in estimators:
    cv_scores = cross_val_score(estimator = model[1], X = train_X, y = train_y, cv = 5, scoring = 'neg_mean_squared_error')
    rmse_scores = np.sqrt(-cv_scores)
    print(f"{model[0]} Cross-Validation Mean: {rmse_scores.mean():.2f} +/- {rmse_scores.std():.2f}\n") 



In [ ]:
# Fit the model on the entire training set and make predictions on the test set
xgb_model.fit(training_values, training_target)
lgb_model.fit(training_values, training_target)
cat_model.fit(training_values, training_target)
voting_reg_model.fit(training_values, training_target)

In [ ]:
# Predict y label on test data
preds_xgb = xgb_model.predict(test_values)
preds_lgb = lgb_model.predict(test_values)
preds_cat = cat_model.predict(test_values)
preds_voting_reg = voting_reg_model.predict(test_values)

### 9. Results
<i>This involves evaluating the final model on a test set and presenting the results, including metrics such as accuracy, precision, recall, and F1 score.</i>


In [ ]:
# Submit prediction to csv
df_test = pd.read_csv("../input/playground-series-s3e8/test.csv")
submission_xgb = pd.DataFrame({'Id': df_test.id, 'price': preds_xgb.astype(float)})
submission_lgb = pd.DataFrame({'Id': df_test.id, 'price': preds_lgb.astype(float)})
submission_cat = pd.DataFrame({'Id': df_test.id, 'price': preds_cat.astype(float)})
submission_voting_reg = pd.DataFrame({'Id': df_test.id, 'price': preds_voting_reg.astype(float)})

submission_xgb.to_csv('submission_xgb.csv', index = False)
submission_lgb.to_csv('submission_lgb.csv', index = False)
submission_cat.to_csv('submission_cat.csv', index = False)
submission_voting_reg.to_csv('submission_voting_reg.csv', index = False)